In [2]:
import numpy as np
import h5py

datafile = h5py.File('SCNeuronModelCompetition.mat')
movie = datafile.get('trainingmovie_norm') # normalized movie
frhist = datafile.get('FRhist_tr') # firing rate histograms
# movie = np.array(movie).reshape(288, 150, 128, 96, 1)
print(movie.shape)

(288, 150, 12288)


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3127302179330207108
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 15927646618
locality {
  bus_id: 1
}
incarnation: 17814482850690332128
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0"
]


In [5]:
import keras
from keras.layers import CuDNNLSTM, Activation, Dense, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Dropout, Reshape
from keras.layers import TimeDistributed

seq_len = movie.shape[1]
input_size = movie.shape[2]
num_layer = 3
num_hidden = 100
output_size = frhist.shape[2]

model = keras.models.Sequential()

model.add(CuDNNLSTM(num_hidden, return_sequences=True, input_shape=(seq_len, input_size)))
# for _ in range(1, num_layer):
#     model.add(BatchNormalization(momentum=0))
#     model.add(Activation('relu'))
#     model.add(CuDNNLSTM(num_hidden, return_sequences=True))

# model.add(BatchNormalization(momentum=0))
# model.add(Activation('linear'))
model.add(TimeDistributed(Dense(output_size)))
model.add(Activation('softplus'))
adamopt = keras.optimizers.Adam(lr=0.001, decay=1e-7)

model.compile(optimizer=adamopt, loss='poisson')
model.summary()

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(movie, frhist, epochs=200, batch_size=32, validation_split=0.2, shuffle=True, callbacks=[early_stopping])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_2 (CuDNNLSTM)     (None, 150, 100)          4956000   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 150, 54)           5454      
_________________________________________________________________
activation_1 (Activation)    (None, 150, 54)           0         
Total params: 4,961,454
Trainable params: 4,961,454
Non-trainable params: 0
_________________________________________________________________
Train on 230 samples, validate on 58 samples
Epoch 1/200
230/230 [==============================] - 6s 26ms/step - loss: 0.5826 - val_loss: 0.4721
Epoch 2/200
230/230 [==============================] - 5s 21ms/step - loss: 0.4421 - val_loss: 0.4054
Epoch 3/200
230/230 [==============================] - 5s 22ms/step - loss: 0.3884 - val_loss: 0.3715
Epoch 4/200
230/230 [========================